## 701 Coursework

In [1]:
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import LabelEncoder
import category_encoders as ce
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score



import matplotlib.pyplot as plt
import seaborn as sns


#print(os.getcwd())
#print(os.listdir())
datafile = './Documents/fraud_oracle.csv'
dataset = pd.read_csv(datafile)
pd.set_option("display.max.columns", None)


dataset.head()

,Month,WeekOfMonth,DayOfWeek,Make,AccidentArea,DayOfWeekClaimed,MonthClaimed,WeekOfMonthClaimed,Sex,MaritalStatus,Age,Fault,PolicyType,VehicleCategory,VehiclePrice,FraudFound_P,PolicyNumber,RepNumber,Deductible,DriverRating,Days_Policy_Accident,Days_Policy_Claim,PastNumberOfClaims,AgeOfVehicle,AgeOfPolicyHolder,PoliceReportFiled,WitnessPresent,AgentType,NumberOfSuppliments,AddressChange_Claim,NumberOfCars,Year,BasePolicy
0,Dec,5,Wednesday,Honda,Urban,Tuesday,Jan,1,Female,Single,21,Policy Holder,Sport - Liability,Sport,more than 69000,0,1,12,300,1,more than 30,more than 30,none,3 years,26 to 30,No,No,External,none,1 year,3 to 4,1994,Liability
1,Jan,3,Wednesday,Honda,Urban,Monday,Jan,4,Male,Single,34,Policy Holder,Sport - Collision,Sport,more than 69000,0,2,15,400,4,more than 30,more than 30,none,6 years,31 to 35,Yes,No,External,none,no change,1 vehicle,1994,Collision
2,Oct,5,Friday,Honda,Urban,Thursday,Nov,2,Male,Married,47,Policy Holder,Sport - Collision,Sport,more than 69000,0,3,7,400,3,more than 30,more than 30,1,7 years,41 to 50,No,No,External,none,no change,1 vehicle,1994,Collision
3,Jun,2,Saturday,Toyota,Rural,Friday,Jul,1,Male,Married,65,Third Party,Sedan - Liability,Sport,20000 to 29000,0,4,4,400,2,more than 30,more than 30,1,more than 7,51 to 65,Yes,No,External,more than 5,no change,1 vehicle,1994,Liability
4,Jan,5,Monday,Honda,Urban,Tuesday,Feb,2,Female,Single,27,Third Party,Sport - Collision,Sport,more than 69000,0,5,3,400,1,more than 30,more than 30,none,5 years,31 to 35,No,No,External,none,no change,1 vehicle,1994,Collision


In [2]:
print(len(dataset))
dataset.shape
dataset.info

15420


<bound method DataFrame.info of       Month  WeekOfMonth  DayOfWeek     Make AccidentArea DayOfWeekClaimed  \
0       Dec            5  Wednesday    Honda        Urban          Tuesday   
1       Jan            3  Wednesday    Honda        Urban           Monday   
2       Oct            5     Friday    Honda        Urban         Thursday   
3       Jun            2   Saturday   Toyota        Rural           Friday   
4       Jan            5     Monday    Honda        Urban          Tuesday   
...     ...          ...        ...      ...          ...              ...   
15415   Nov            4     Friday   Toyota        Urban          Tuesday   
15416   Nov            5   Thursday  Pontiac        Urban           Friday   
15417   Nov            5   Thursday   Toyota        Rural           Friday   
15418   Dec            1     Monday   Toyota        Urban         Thursday   
15419   Dec            2  Wednesday   Toyota        Urban         Thursday   

      MonthClaimed  WeekOfMonth

In [3]:

dataset.describe(include=object)

,Month,DayOfWeek,Make,AccidentArea,DayOfWeekClaimed,MonthClaimed,Sex,MaritalStatus,Fault,PolicyType,VehicleCategory,VehiclePrice,Days_Policy_Accident,Days_Policy_Claim,PastNumberOfClaims,AgeOfVehicle,AgeOfPolicyHolder,PoliceReportFiled,WitnessPresent,AgentType,NumberOfSuppliments,AddressChange_Claim,NumberOfCars,BasePolicy
count,15420,15420,15420,15420,15420,15420,15420,15420,15420,15420,15420,15420,15420,15420,15420,15420,15420,15420,15420,15420,15420,15420,15420,15420
unique,12,7,19,2,8,13,2,4,2,9,3,6,5,4,4,8,9,2,2,2,4,5,5,3
top,Jan,Monday,Pontiac,Urban,Monday,Jan,Male,Married,Policy Holder,Sedan - Collision,Sedan,20000 to 29000,more than 30,more than 30,2 to 4,7 years,31 to 35,No,No,External,none,no change,1 vehicle,Collision
freq,1411,2616,3837,13822,3757,1446,13000,10625,11230,5584,9671,8079,15247,15342,5485,5807,5593,14992,15333,15179,7047,14324,14316,5962


In [4]:
dataset.isna().values.any()

False

In [5]:
cols = dataset.columns.tolist()
print("Columns -> ",cols,"\n\n")
print("Unique values in each column\n")
for i in cols:
    print(dataset[i].unique())

Columns ->  ['Month', 'WeekOfMonth', 'DayOfWeek', 'Make', 'AccidentArea', 'DayOfWeekClaimed', 'MonthClaimed', 'WeekOfMonthClaimed', 'Sex', 'MaritalStatus', 'Age', 'Fault', 'PolicyType', 'VehicleCategory', 'VehiclePrice', 'FraudFound_P', 'PolicyNumber', 'RepNumber', 'Deductible', 'DriverRating', 'Days_Policy_Accident', 'Days_Policy_Claim', 'PastNumberOfClaims', 'AgeOfVehicle', 'AgeOfPolicyHolder', 'PoliceReportFiled', 'WitnessPresent', 'AgentType', 'NumberOfSuppliments', 'AddressChange_Claim', 'NumberOfCars', 'Year', 'BasePolicy'] 


Unique values in each column

['Dec' 'Jan' 'Oct' 'Jun' 'Feb' 'Nov' 'Apr' 'Mar' 'Aug' 'Jul' 'May' 'Sep']
[5 3 2 4 1]
['Wednesday' 'Friday' 'Saturday' 'Monday' 'Tuesday' 'Sunday' 'Thursday']
['Honda' 'Toyota' 'Ford' 'Mazda' 'Chevrolet' 'Pontiac' 'Accura' 'Dodge'
 'Mercury' 'Jaguar' 'Nisson' 'VW' 'Saab' 'Saturn' 'Porche' 'BMW' 'Mecedes'
 'Ferrari' 'Lexus']
['Urban' 'Rural']
['Tuesday' 'Monday' 'Thursday' 'Friday' 'Wednesday' 'Saturday' 'Sunday'
 '0']
['Jan' 'N

In [6]:
df1 = dataset.groupby('DayOfWeekClaimed').agg({"DayOfWeekClaimed":"count"})
print(df1)

df2 = dataset.groupby('MonthClaimed').agg({"MonthClaimed": "count"})
print(df2)

df3 = dataset.groupby('Age').agg({"Age": "count"})
print(df3)


#dataset = dataset[dataset.DayOfWeekClaimed!=0]

                  DayOfWeekClaimed
DayOfWeekClaimed                  
0                                1
Friday                        2497
Monday                        3757
Saturday                       127
Sunday                          52
Thursday                      2660
Tuesday                       3375
Wednesday                     2951
              MonthClaimed
MonthClaimed              
0                        1
Apr                   1271
Aug                   1126
Dec                   1146
Feb                   1287
Jan                   1446
Jul                   1225
Jun                   1293
Mar                   1348
May                   1411
Nov                   1285
Oct                   1339
Sep                   1242
     Age
Age     
0    320
16     9
17     6
18    48
19    32
..   ...
76    42
77    29
78    35
79    20
80    32

[66 rows x 1 columns]


In [7]:
dataset = dataset[dataset.DayOfWeekClaimed!=0]
dataset = dataset[dataset.MonthClaimed!=0]
dataset = dataset[dataset.Age!=0]

In [8]:

#print(dataset['Fault'].dtype)
dfp = dataset['Fault'].replace({"Policy Holder": 1, "Third Party": 2})
dataset['Fault'] = dfp

dfp = dataset['DayOfWeek'].replace({"Monday": 1, "Tuesday": 2, "Wednesday": 3, "Thursday": 4, "Friday": 5, "Saturday": 6, "Sunday": 7 })
dataset['DayOfWeek'] = dfp
#print(dataset['Fault'].dtype)

dfp = dataset['AccidentArea'].replace({"Urban": 1, "Rural": 2})
dataset['AccidentArea'] = dfp

dfp = dataset['DayOfWeekClaimed'].replace({"Monday": 1, "Tuesday": 2, "Wednesday": 3, "Thursday": 4, "Friday": 5, "Saturday": 6, "Sunday": 7 })
dataset['DayOfWeekClaimed'] = dfp

dfp = dataset['PoliceReportFiled'].replace({"No": 0, "Yes": 1})
dataset['PoliceReportFiled'] = dfp

dfp = dataset['WitnessPresent'].replace({"No": 0, "Yes": 1})
dataset['WitnessPresent'] = dfp

dfp = dataset['AgentType'].replace({"External": 1, "Internal": 2})
dataset['AgentType'] = dfp

#replacing the categorical value 2 to 4 with mean 3 and more than 4 with arbitrary value 7
dfp = dataset['PastNumberOfClaims'].replace({"none": 0, "1": 1, "2 to 4": 3, "more than 4": 7})
dataset['PastNumberOfClaims'] = dfp

dfp = dataset['Make'].replace({"Honda": 1, "Toyota": 2, "Ford": 3, "Mazda": 4, "Chevrolet": 5, "Pontiac": 6, "Accura": 7, "Dodge": 8,
 "Mercury": 9, "Jaguar": 10, "Nisson": 11, "VW": 12, "Saab": 13, "Saturn": 14, "Porche": 15, "BMW":16, "Mecedes": 17,
 "Ferrari": 18, "Lexus": 19 })
dataset['Make'] = dfp

dfp = dataset['VehicleCategory'].replace({"Sport": 1, "Utility": 2, "Sedan": 3})
dataset['VehicleCategory'] = dfp


dfp = dataset['AgeOfVehicle'].replace({"new": 0.1, "2 years": 2.0, "3 years": 3.0, "4 years": 4.0, "5 years": 5.0, "6 years": 6.0, "7 years": 7, "more than 7": 8})
dataset['AgeOfVehicle'] = dfp


dfp = dataset['Days_Policy_Accident'].replace({"none": 0, "1 to 7": 4, "8 to 15": 12, "15 to 30": 23, "more than 30": 36 })
dataset['Days_Policy_Accident'] = dfp

dfp = dataset['Days_Policy_Claim'].replace({"none": 0, "8 to 15": 12, "15 to 30": 23, "more than 30": 36 })
dataset['Days_Policy_Claim'] = dfp

dfp = dataset['NumberOfCars'].replace({"1 vehicle": 1, "2 vehicles": 2, "3 to 4": 4, "5 to 8": 7, "more than 8": 9 })
dataset['NumberOfCars'] = dfp


 
dfp1 = dataset['PolicyType']

dfp =  dataset['PolicyType'].replace({"Sport - Liability":"Liability", "Sport - Collision": "Collision", "Sedan - Liability": "Liability",
 "Utility - All Perils": "All Perils", "Sedan - All Perils": "All Perils", "Sedan - Collision": "Collision",
 "Utility - Collision": "Collision", "Utility - Liability": "Liability", "Sport - All Perils": "All Perils" })


 
print("Columns PolicyType and BasePolicy are the same? ",dfp.equals(dataset['BasePolicy']))

dfp =  dataset['PolicyType'].replace({"Sport - Liability": 1, "Sport - Collision": 1, "Sedan - Liability": 3,
 "Utility - All Perils": 2, "Sedan - All Perils": 3, "Sedan - Collision": 3,
 "Utility - Collision": 2, "Utility - Liability": 2, "Sport - All Perils": 1 })

dataset['PolicyType'] = dfp

dfp = dataset['BasePolicy'].replace({"Liability": 1, "Collision": 2, "All Perils": 3})
dataset['BasePolicy'] = dfp

#1 year' 'no change' '4 to 8 years' '2 to 3 years' 'under 6 months: using max values for this case
dfp = dataset['AddressChange_Claim'].replace({"no change": 0, "under 6 months": 0.5, "1 year": 1, "2 to 3 years": 3, "4 to 8 years": 8})
dataset['AddressChange_Claim'] = dfp

#none' 'more than 5' '3 to 5' '1 to 2
dfp = dataset['NumberOfSuppliments'].replace({"none": 0, "1 to 2": 2, "3 to 5": 4, "more than 5": 6})
dataset['NumberOfSuppliments'] = dfp

#more than 69000' '20000 to 29000' '30000 to 39000' 'less than 20000' '40000 to 59000' '60000 to 69000
dfp = dataset['VehiclePrice'].replace({"less than 20000": 15000, "20000 to 29000": 25000, "30000 to 39000": 35000, "40000 to 59000": 50000, "60000 to 69000": 65000, "more than 69000": 80000})
dataset['VehiclePrice'] = dfp

#26 to 30' '31 to 35' '41 to 50' '51 to 65' '21 to 25' '36 to 40' '16 to 17' 'over 65' '18 to 20
dfp = dataset['AgeOfPolicyHolder'].replace({"16 to 17": 17, "18 to 20": 20, "21 to 25": 23, "26 to 30": 28, "31 to 35": 33, "36 to 40": 38, "41 to 50": 45, "51 to 65": 58, "over 65": 72})
dataset['AgeOfPolicyHolder'] = dfp




print("Columns PolicyType and BasePolicy are the same? ",dataset['Days_Policy_Accident'].equals(dataset['Days_Policy_Claim']))



#dataset.drop('PolicyNumber', axis = 1, inplace = True)

dataset.head()

Columns PolicyType and BasePolicy are the same?  True
Columns PolicyType and BasePolicy are the same?  False


,Month,WeekOfMonth,DayOfWeek,Make,AccidentArea,DayOfWeekClaimed,MonthClaimed,WeekOfMonthClaimed,Sex,MaritalStatus,Age,Fault,PolicyType,VehicleCategory,VehiclePrice,FraudFound_P,PolicyNumber,RepNumber,Deductible,DriverRating,Days_Policy_Accident,Days_Policy_Claim,PastNumberOfClaims,AgeOfVehicle,AgeOfPolicyHolder,PoliceReportFiled,WitnessPresent,AgentType,NumberOfSuppliments,AddressChange_Claim,NumberOfCars,Year,BasePolicy
0,Dec,5,3,1,1,2,Jan,1,Female,Single,21,1,1,1,80000,0,1,12,300,1,36,36,0,3.0,28,0,0,1,0,1.0,4,1994,1
1,Jan,3,3,1,1,1,Jan,4,Male,Single,34,1,1,1,80000,0,2,15,400,4,36,36,0,6.0,33,1,0,1,0,0.0,1,1994,2
2,Oct,5,5,1,1,4,Nov,2,Male,Married,47,1,1,1,80000,0,3,7,400,3,36,36,1,7.0,45,0,0,1,0,0.0,1,1994,2
3,Jun,2,6,2,2,5,Jul,1,Male,Married,65,2,3,1,25000,0,4,4,400,2,36,36,1,8.0,58,1,0,1,6,0.0,1,1994,1
4,Jan,5,1,1,1,2,Feb,2,Female,Single,27,2,1,1,80000,0,5,3,400,1,36,36,0,5.0,33,0,0,1,0,0.0,1,1994,2


In [9]:
df1 = dataset.groupby('DayOfWeekClaimed').agg({"DayOfWeekClaimed":"count"})
print(df1)

df2 = dataset.groupby('MonthClaimed')['MonthClaimed'].count()
print(df2)

df3 = dataset.groupby('Age').agg({"Age": "count"})
print(df3)


#plt.hist(df2['MonthClaimed'])

                  DayOfWeekClaimed
DayOfWeekClaimed                  
1                             3684
2                             3297
3                             2890
4                             2616
5                             2434
6                              127
7                               52
MonthClaimed
Apr    1217
Aug    1118
Dec    1139
Feb    1240
Jan    1375
Jul    1216
Jun    1271
Mar    1298
May    1376
Nov    1276
Oct    1337
Sep    1237
Name: MonthClaimed, dtype: int64
     Age
Age     
16     9
17     6
18    48
19    32
20    28
..   ...
76    42
77    29
78    35
79    20
80    32

[65 rows x 1 columns]


In [17]:
y = dataset['FraudFound_P']
y

0        0
1        0
2        0
3        0
4        0
        ..
15415    1
15416    0
15417    1
15418    0
15419    1
Name: FraudFound_P, Length: 15100, dtype: int64